In [ ]:
# Import packages
import ee
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize()

# Define ROI for Punjab, India
roi = ee.Geometry.Rectangle([74.5, 29.5, 76.5, 31.5])

# Load Sentinel-2 collection
collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(roi) \
    .filterDate('2023-01-01', '2023-03-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Apply cloud masking
def mask_clouds(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)
    return image.updateMask(cloud_mask)

collection = collection.map(mask_clouds)

# Calculate NDVI
def calculate_ndvi(image):
    nir = image.select('B8')  # Near-Infrared band
    red = image.select('B4')  # Red band
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')  # NDVI formula
    return image.addBands(ndvi)

ndvi_collection = collection.map(calculate_ndvi)

# Export NDVI image (median composite)
task = ee.batch.Export.image.toDrive(
    image=ndvi_collection.median(),  # Median composite of the collection
    description='NDVI_Punjab_2023',
    scale=10,  # 10m resolution
    region=roi,
    fileFormat='GeoTIFF',
    maxPixels=1e13
)

# Start the export task
task.start()

# Check task status
print(task.status())